# Introduction
## **WELCOME!!👋👋**

This notebook is the first exercise in the Learning Path for the Maven Course: [Build Multi-Agent Applications - A Bootcamp](https://maven.com/aggregate-intellect/llm-systems). Let’s get started on developing our first LLM powered application together!


## Objective 🎯:

The goal is to create a LLM Application that:
- Answer question about best practices of RAG as defined in AISC Substack Article: [How to do retrieval augmented generation (RAG) right!](https://aisc.substack.com/p/how-to-do-retrieval-augmented-generation)
- Develop a front-end UI using Chainlit


# Libraries Required for the Excercise

In [1]:
%%capture --no-stderr
! pip install langsmith langchain-community langchain chromadb tiktoken langchain_openai chainlit
!npm install localtunnel

## Setup Required - APIs

In this notebook you'll be using OpenAI models for reasoning and generation, langsmith for tracing and langchain for orchestration, chainlit for UI.

In [3]:
import os
from langsmith import utils
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

import openai
from langsmith import traceable

import getpass

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# define your langchain project name here
os.environ["LANGCHAIN_PROJECT"] = "maven-course-learning-path"

In [5]:
def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"Enter your {key}:")


_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")

Enter your OPENAI_API_KEY:··········
Enter your LANGCHAIN_API_KEY:··········


In [6]:
print('Is Tracing is enabled:', utils.tracing_is_enabled())

Is Tracing is enabled: True


## Data Processing

In [7]:
### INDEX

from bs4 import BeautifulSoup as Soup
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# Load
url = "https://aisc.substack.com/p/how-to-do-retrieval-augmented-generation"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Index
retriever = vectorstore.as_retriever()

## RAG Application

In [8]:
# RAG
class RagApp:

    def __init__(self, retriever, model: str = "gpt-4o-mini", prompt: str = "rlm/rag-prompt"):
        self._retriever = retriever
        self._llm = ChatOpenAI(model = model)
        # Prompt Link: https://smith.langchain.com/hub/rlm/rag-prompt)
        self._prompt = hub.pull(prompt)

    @traceable()
    def retrieve_docs(self, question):
        return self._retriever.invoke(question)

    @traceable()
    def get_answer(self, question: str):
        rag_chain = (
            {"context": self._retriever, "question": RunnablePassthrough()}
            | self._prompt
            | self._llm
        )

        response = rag_chain.invoke(question)
        similar = self.retrieve_docs(question)
        #Evaluators will expect "answer" and "contexts"
        return {
            "answer": response.content,
            "contexts": [str(doc) for doc in similar],
        }

rag_bot = RagApp(retriever)

In [9]:
rag_response = rag_bot.get_answer("Name all individuals whose presentation insipred this blog")

In [10]:
rag_response

{'answer': 'The individuals whose presentations inspired the blog are Pai, Ian Yu, Nikhil Varghese, Percy Chen, and Suhas.',
 'contexts': ['page_content=\'Pai, Ian Yu, Nikhil Varghese, and Percy Chen. Some of the thought processes presented here are borrowed with permission from the content of Suhas’s book, Designing Large Language Models Applications. The early drafts of this article and the accompanying visuals were provided by Mohsin Iqbal.Thanks\' metadata={\'content_type\': \'text/html; charset=utf-8\', \'description\': \'My client: “hey, we have this vector db + LLM RAG thing and it’s not working”. And my answer is often “pull in the chair and sit down, we need to talk about how robust software is built".\', \'language\': \'en\', \'source\': \'https://aisc.substack.com/p/how-to-do-retrieval-augmented-generation\', \'title\': \'How to do retrieval augmented generation (RAG) right!\'}',
  'page_content=\'Jul 10Thanks for the article and your speech at AI Practioner event! Expand fu

## Create Front-end with Chainlit

In [11]:
%%writefile app.py
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
import chainlit as cl

from bs4 import BeautifulSoup
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
import os

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def get_retriever():
    # Load
    url = "https://aisc.substack.com/p/how-to-do-retrieval-augmented-generation"
    loader = RecursiveUrlLoader(
        url=url,
        max_depth=10,
        extractor=lambda x: BeautifulSoup(x, "html.parser").text
    )
    docs = loader.load()

    # Split
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)

    # Embed
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

    # Return retriever with search kwargs
    return vectorstore.as_retriever(search_kwargs={"k": 3})

@cl.on_chat_start
async def start():
    # Initialize the chain components
    retriever = get_retriever()
    llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)
    prompt = hub.pull('rlm/rag-prompt')
    # Create the chain
    rag_chain = (
        {
            "context": lambda x: format_docs(retriever.get_relevant_documents(x["question"])),
            "question": RunnablePassthrough()
        }
        | prompt
        | llm
    )

    # Store the chain in the user session
    cl.user_session.set("chain", rag_chain)

    # Send an initial message
    await cl.Message(content="Hi! I'm ready to help you with questions about AISC substack article on RAG. What would you like to know?").send()

@cl.on_message
async def main(message: cl.Message):
    chain = cl.user_session.get("chain")  # Get the chain from the session

    if chain is None:
        await cl.Message(content="Error: Chain not initialized. Please restart the chat.").send()
        return

    msg = cl.Message(content="")

    async with cl.Step("Fetched Content 🤔"):
        async for chunk in chain.astream(
            {"question": message.content},
            config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
        ):
            await msg.stream_token(chunk.content)

    await msg.send()

if __name__ == "__main__":
    cl.run()

Writing app.py


### Access Chainlit service running on the colab instance using [localtunnel](https://github.com/localtunnel/localtunnel)

#### You'll need to enter the following password to access the dashboard

In [12]:
print('your password is: ')
!curl ipecho.net/plain

your password is: 
34.46.111.8

In [13]:
!chainlit run app.py & npx localtunnel --port 8000

2024-11-23 07:31:15 - Created default config file at /content/.chainlit/config.toml
2024-11-23 07:31:15 - Created default translation directory at /content/.chainlit/translations
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/ta.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/he-IL.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/gu.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/kn.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/hi.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/mr.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/ml.json
2024-11-23 07:31:15 - Created default translation file at /content/.chainlit/translations/en-US.json
2024-11-23 07:31:15 - Created d